In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import datetime
import io
import sqlite3

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(chrome_options=options)

<ipython-input-2-b5f1867c00b1>:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


In [3]:
driver.get("https://www.google.com")

In [4]:
button=driver.find_element_by_xpath("//input")

In [5]:
button.send_keys("google play pinterest")

In [6]:
button.send_keys(Keys.RETURN)

In [7]:
#serach for the applcation in the Google play
try:
    link=WebDriverWait(driver,10).until(
        expected_conditions.presence_of_all_elements_located(
            (By.PARTIAL_LINK_TEXT, "Pinterest – Apps on Google Play")
        )                
    )
    link[0].click()
except Exception as e:
    print(e)

In [8]:
#click on the read all revies for the first time in the page
try:
    read_all_reviews=WebDriverWait(driver, 4).until(
        expected_conditions.presence_of_element_located((By.XPATH, '//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span'))
    )  
except Exception as e:
    print(e)
read_all_reviews.click()
#//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[6]/div/span/span

In [9]:
def create_table_pintrest():
    conn = sqlite3.connect(database='hesam.db')
    cur = conn.cursor()
    cur.execute(
        # 'comment', 'author', 'rate', 'cr_date','likes'
        'CREATE TABLE IF NOT EXISTS [pintrest]  (comment text,author text, rate int,cr_date text, likes int  )')
    conn.commit()
    conn.close()
# insert

def insert(comment ,author , rate ,cr_date , likes):
    conn = sqlite3.connect(database='hesam.db')
    cur = conn.cursor()
    cur.execute("insert into pintrest values(?,?,?,?,?)", (comment ,author , rate ,cr_date , likes ))
    conn.commit()
    conn.close()
    
def select():
    conn = sqlite3.connect('hesam.db')
    cur = conn.cursor()
    cur.execute('select * from pintrest')
    t1 = cur.fetchall()
    conn.close()
    return t1

def delete():
    conn = sqlite3.connect('hesam.db')
    cur = conn.cursor()
    cur.execute("delete from pintrest ")
    conn.commit()
    conn.close()

In [10]:
create_table_pintrest()

In [11]:
delete()

In [12]:
def ful_reviws(i):
    result ='ful_reviws'
    try:
        
        ful_reviws=driver.find_element_by_xpath(
            F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span[1]/div/button'
            )
        ful_reviws.click()

    except :
        pass

def get_comments(i):
    result='get_comments'
     
    try:
        result= driver.find_elements_by_xpath(
                F'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[2]/span'
                )
        result=''.join ([x.text for x in result ])
        
    except Exception as e:
        print(e) 
    return result

def get_author(i):
    result='get_author'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/span'
        ).text
    except Exception as e:
        print(e) 
    return result 

def get_rating(i):
    result='get_rating'
    try:
        rating=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[1]/div/div'
        ).get_attribute("aria-label")
        result =rating [6:7]
    except Exception as e:
        print(e) 
    return result 

def get_date(i):
    result='get_date'
    try:
        date=driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div/main/div/div[1]/div[2]/div/div[{i}]/div/div[2]/div[1]/div[1]/div/span[2]'
        ).text
        result=  datetime.datetime.strptime( date.replace(',',''), '%B %d %Y')
    except Exception as e:
        print(e) 
    return result 

def get_like(i):
    result='get_like'
    try:
        result= driver.find_element_by_xpath(
            f'//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div/main/div/div[1]/div[2]/div[1]/div[{i}]/div/div[2]/div[1]/div[2]/div/div[1]/div[2]'
        ).text
    except Exception as e:
        print(e) 
    return result 
             

In [13]:
#show complete comments in the pfirst page 
#Todo : compete this part for the future use in the loop part 

def get_info(i):
    #try to pen comments
    ful_reviws(i)
    #get comments
    comment=get_comments(i)
    author=get_author(i)
    rating=get_rating(i)
    date=get_date(i)
    likes=get_like(i)

    # print(f"author:{author} rate: {rating} cr_date: {date} likes: {likes}")
    insert(comment,author,rating ,date , likes  )




In [14]:
i=0
j=1
while i<200:
    j+=1
    i+=1
    try:
        showmore=WebDriverWait(driver, 2).until(
            expected_conditions.presence_of_element_located(
                (By.XPATH,'//div[@class="PFAhAf"]/div[@role="button"]/span[@class="CwaK9"]/span[@class="RveJvd snByac"]')
            )
        )
        showmore.click()
    except Exception as  e:
        pass
    finally:
        time.sleep(1)
        body = driver.find_element_by_css_selector('body')
    
        while j % 50 != 0:
            get_info(j)
            if j % 10 == 0 :
                for zi in range((j//10)+5):
                    body.send_keys(Keys.PAGE_DOWN)
                    time.sleep(.2)

            j+=1
        time.sleep(.5)



NameError: name 'I' is not defined

In [20]:
# delete()

In [ ]:
# 